# Orbit Bias Analysis

### Compare mean/median discharge from FS vs SO to check for systematic differences

In [ ]:
# Merge FS reaches into its own list

# Credit to Nikki Tebaldi, https:/podaac.github.io/tutorials/notebooks/datasets/SWOT_L4_DAWG_SOS_DISCHARGE.html#plot-discharge-timeseries
import datetime
import time
import pathlib
import os,sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import netCDF4 as nc
import numpy as np
import pandas as pd
import cartopy
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import folium
import requests
from io import StringIO
import h5py
import math
import zipfile
from datetime import timedelta
import json
import seaborn as sns 
import warnings
from tqdm import tqdm 
from itertools import islice
from scipy.stats import pearsonr
from itertools import combinations
from scipy.stats import genextreme
from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel
from scipy import stats
from scipy.stats import median_abs_deviation, shapiro, ttest_1samp, wilcoxon
from scipy.stats import ks_2samp, wasserstein_distance, shapiro, wilcoxon
from sklearn.metrics import mean_squared_error

#Specialized Functions
from swotOrbitFunctions_ungauged import *
divide_date = pd.to_datetime('2023-07-11')


plt.rcParams['font.family'] = 'serif'

plt.rcParams.update({
    'axes.titlesize': 30,    # Title font size
    'axes.labelsize': 22,     # Axis labels font size
    'xtick.labelsize': 20,    # X-axis ticks font size
    'ytick.labelsize': 20,    # Y-axis ticks font size
    'legend.fontsize': 20,    # Legend font size
    'font.size': 20,          # Global font size for text
    'figure.titlesize': 24,   # Figure title font size
    'lines.linewidth': 2,     # Line width
    'axes.linewidth': 2,      # Axis line width
    'axes.grid': True,        # Show grid
    'grid.linestyle': '--',   # Dashed grid lines
    'grid.alpha': 0.5,        # Grid line transparency
    'figure.figsize': (10, 6) # Figure size (width, height in inches)
})



def get_season_orbits(date):
    month = date.month
    day = date.day
    if (month == 3 and day >= 30) or (month in [4, 5, 6]) or (month == 7 and day < 12):
        return '4-7'
    elif (month == 7 and day >= 12) or (month in [7, 8]) or (month == 9 and day < 30):
        return '7-10'
    elif (month == 9 and day >= 30) or (month in [10, 11]):
        return '10-1'
    else:
        return '1-4'

color_dict = {
    'sic4dvar': 'green',
    'momma': 'blue',
    'neobam': 'purple',
    'consensus': 'sienna',
    'metroman': 'orange',
    'geobam': 'purple',
    'hivdi': 'deeppink',
    'sad': 'tomato',
    'gauge': 'black'
}

def tukey_filter(df, column, threshold=1.5):
    """
    Apply Tukey outlier filtering to a specific column in the DataFrame.
    
    Parameters:
    - df: pandas DataFrame
    - column: column name (string) to apply Tukey filter on
    - threshold: float, the multiplier for the IQR (default is 1.5)
    
    Returns:
    - Filtered DataFrame where values are within [Q1 - threshold*IQR, Q3 + threshold*IQR]
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

def calc_cons(df):
    df = df[df['time_str'] != 'no_data']
    if 'consensus' not in df['algo'].unique():
            algo_Q_cons_values = (
                df[df['algo'] != 'gauge']
                .groupby(['reach_id', 'time'])['Q']
                .median()
                .reset_index()
            )
            algo_Q_cons_values['algo'] = 'consensus'
            df = pd.concat([df, algo_Q_cons_values], ignore_index=True)
    return df


def modified_z_filter(series, threshold=3.5):
    median = np.median(series)
    mad = median_abs_deviation(series, scale='normal')
    if mad == 0:
        return series
    mzs = 0.6745 * (series - median) / mad
    return series[np.abs(mzs) <= threshold]

## Load Reach Discharge

In [ ]:
#reach hydrograph comparison

runC_perm = pd.read_csv('/ContinuousRun').drop_duplicates(subset=['reach_id','algo','time_str'])

runB_perm = pd.read_csv('/ScienceRun').drop_duplicates(subset=['reach_id','algo','time_str']) #runB_124_perm,  ,runC_6789_perm

runA_perm = pd.read_csv('/FastRun').drop_duplicates(subset=['reach_id','algo','time_str']) #runB_124_perm,  ,runC_6789_perm

runE_perm = pd.read_csv('/SampledRun').drop_duplicates(subset=['reach_id','algo','time_str']).dropna(subset='time_str')

print(runA_perm.groupby(['algo']).reach_id.nunique())
print(runB_perm.groupby(['algo']).reach_id.nunique())
print(runC_perm.groupby(['algo']).reach_id.nunique())
print(runE_perm.groupby(['algo']).reach_id.nunique())


In [ ]:
#Consensus and other Cleaning

# Ensure 'time' column is in datetime format
runC_perm['time'] = pd.to_datetime(runC_perm['time'])
runC_perm = runC_perm[(runC_perm['Q'] > 1) & (runC_perm['Q'] < 1e7)]

# Define the cutoff date
divide_date = pd.Timestamp("2023-07-11")

# First, filter for 'algo' == 'consensus'
runC_perm_consensus = runC_perm[runC_perm['algo'] == 'consensus']

# Identify reach_ids that have at least 10 observations before and after the cutoff
valid_reach_ids = runC_perm_consensus.groupby('reach_id').filter(
    lambda g: (g['time'] < divide_date).sum() >= 10 and (g['time'] >= divide_date).sum() >= 10
)['reach_id'].unique()

# Apply this filter to the full dataset (all algorithms included)
runC_perm_clean = runC_perm[runC_perm['reach_id'].isin(valid_reach_ids)]

# Print the number of unique reaches
print('# of unique reaches runC with min 10 values: ', runC_perm_clean['reach_id'].nunique())


valid_reach_ids_C_min = runC_perm_consensus.groupby('reach_id').filter(
    lambda g: (g['time'] < divide_date).sum() >= 1 and (g['time'] >= divide_date).sum() >= 1
)['reach_id'].unique()

# Apply this filter to the full dataset (all algorithms included)
runC_perm_clean_min = runC_perm[runC_perm['reach_id'].isin(valid_reach_ids_C_min)]

# Print the number of unique reaches
print('# of unique reaches runC MIN 1 IN FS: ', runC_perm_clean_min['reach_id'].nunique())

#RunA
runA_perm['time'] = pd.to_datetime(runA_perm['time'])
runA_perm = runA_perm[(runA_perm['Q'] > 1) & (runA_perm['Q'] < 1e7)]

runA_perm_consensus = runA_perm[runA_perm['algo'] == 'consensus']

valid_reach_ids_A = runA_perm_consensus.groupby('reach_id').filter(
    lambda g: (g['time'] <= divide_date).sum() >= 10
)['reach_id'].unique()
runA_perm_clean = runA_perm[runA_perm['reach_id'].isin(valid_reach_ids_A)]
print('# of unique reaches A with min 10 values: ', runA_perm_clean['reach_id'].nunique())

#RUN B
runB_perm['time'] = pd.to_datetime(runB_perm['time'])
runB_perm = runB_perm[(runB_perm['Q'] > 1) & (runB_perm['Q'] < 1e7)]

runB_perm_consensus = runB_perm[runB_perm['algo'] == 'consensus']

valid_reach_ids_B = runB_perm_consensus.groupby('reach_id').filter(
    lambda g: (g['time'] >= divide_date).sum() >= 10
)['reach_id'].unique()

runB_perm_clean = runB_perm[runB_perm['reach_id'].isin(valid_reach_ids_B)]
runB_perm_clean = runB_perm_clean[runB_perm_clean['reach_id'].isin(valid_reach_ids_A)]

print('# of unique reaches runB with min 10 values: ', runB_perm_clean['reach_id'].nunique())


#RunD 
runE_perm = runE_perm.dropna(subset=['time'])

runE_perm['time'] = pd.to_datetime(runE_perm['time'])
runE_perm = runE_perm[(runE_perm['Q'] > 1) & (runE_perm['Q'] < 1e7)]

runE_perm_consensus = runE_perm[runE_perm['algo'] == 'consensus']
valid_reach_ids_D = runE_perm_consensus.groupby('reach_id').filter(
    lambda g: (g['time'] <= divide_date).sum() >= 5
)['reach_id'].unique()

runE_perm_clean = runE_perm[runE_perm['reach_id'].isin(valid_reach_ids_D)]
print('# of unique reaches E: ', runE_perm_clean['reach_id'].nunique())


In [ ]:
runC_perm_clean_cont = runC_perm_clean.copy()
runC_perm_clean_cont['algo'] = runC_perm_clean_cont['algo'].replace({'geobam': 'neobam'})

continent_map = {
    '1': 'AF', '2': 'EU', '3': 'SI', '4': 'AS',
    '5': 'OC', '6': 'SA', '7': 'NA', '8': 'AR', '9': 'GR'
}

runC_perm_clean_cont['continent'] = runC_perm_clean_cont['reach_id'].astype(str).str[0]
runC_perm_clean_cont['continent_name'] = runC_perm_clean_cont['continent'].map(continent_map)


# Step 2: Group by algo and continent, then count unique reach_ids
result = runC_perm_clean_cont.groupby(['algo', 'continent_name'])['reach_id'].nunique().reset_index()


# Pivot the data for plotting (so continent_name is index, algos as columns)
pivot_df = result.pivot(index='continent_name', columns='algo', values='reach_id').fillna(0)

# Plotting
ax = pivot_df.plot(kind='bar', figsize=(10, 6), color=[color_dict.get(algo, '#333333') for algo in pivot_df.columns])

#plt.title('Unique reach_id Counts per Continent by Algo')
plt.xlabel('Continent')
plt.ylabel('Frequency (Unique reach_id)')
plt.xticks(rotation=45)
ax.legend(title='Algorithm', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.tight_layout()
plt.show()

In [ ]:
#Subset to common reach_ids

def subset_to_common_reach_ids(dfs, reach_id_col='reach_id', algo_col='algo', consensus_value='consensus'):
    """
    Finds common reach_ids across all DataFrames in the list where the algo column equals 'consensus',
    and subsets each DataFrame to only those reach_ids.

    Parameters:
        dfs (list of pd.DataFrame): List of DataFrames to compare and subset.
        reach_id_col (str): Name of the column that contains reach IDs.
        algo_col (str): Name of the algorithm-identifying column.
        consensus_value (str): The value in algo_col that marks consensus rows.

    Returns:
        list of pd.DataFrame: Subsetted DataFrames with only common consensus reach_ids.
    """
    # Step 1: Find the set of common consensus reach_ids
    common_ids = set(
        dfs[0].loc[dfs[0][algo_col] == consensus_value, reach_id_col]
    )
    for df in dfs[1:]:
        consensus_ids = set(df.loc[df[algo_col] == consensus_value, reach_id_col])
        common_ids &= consensus_ids

    # Step 2: Subset each DataFrame to those reach_ids
    dfs_subset = [df[df[reach_id_col].isin(common_ids)].copy() for df in dfs]
    #Save common reach ids
    with open('common_reaches_abcd.json', 'w') as f:
        json.dump(list(common_ids), f, indent=2)
    
    return dfs_subset



In [ ]:
# Compile for comparison analysis, all function input a dictionary of runs

df_list = [runA_perm_clean, runB_perm_clean, runC_perm_clean] #, runE_perm_clean] #, runD_perm_clean] 
df_list_common = subset_to_common_reach_ids(df_list)
print(df_list_common[0].reach_id.nunique())


dfs_q = {'Fast': df_list_common[0], 'Science': df_list_common[1], 'Continuous': df_list_common[2], 'Sampled': runE_perm_clean}

for label, df in dfs_q.items():
    df['algo'] = df['algo'].replace({'geobam': 'neobam'})
    print(label, df.reach_id.nunique())

In [ ]:
#Calc CV and RMD

dfs_q =append_RMD(dfs_q=dfs_q)
dfs_q = append_coeffVar(dfs_q=dfs_q)
for label, df in dfs_q.items():
    print(label, df.reach_id.nunique())
# cdfPlot_RMD(dfs_q=dfs_q, color_dict=color_dict, column_to_plot = 'RMD_cons')
# #cdfPlot_RMD(dfs_q=dfs_q, color_dict=color_dict, column_to_plot = 'RMD_gauge')
# plot_cdf_coeff(dfs_q=dfs_q, color_dict=color_dict)

In [ ]:

# # Count how many algos each reach_id has
# algo_counts_per_reach = dfs_q['Continuous'].groupby('reach_id')['algo'].nunique()

# Count frequency per algorithm
algo_counts = dfs_q['Continuous'].groupby('algo')['reach_id'].nunique()
# Plot
plt.figure(figsize=(10,6))
bars = plt.bar(algo_counts.index, algo_counts.values, color=[color_dict[a] for a in algo_counts.index])

# Add counts on top of bars
for bar in bars:
    plt.text(
        bar.get_x() + bar.get_width()/2, 
        bar.get_height(), 
        str(int(bar.get_height())), 
        ha='center', va='bottom'
    )

plt.ylim([0, 12500])
plt.ylabel('Number of Reaches')
plt.xlabel('Algorithm')
plt.title('Algorithm Frequency (Continuous run)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
dfs_q = remove_low_cv_and_recalc_consensus(dfs_q, CV_thresh = 0.25)


In [ ]:
for label, df in dfs_q.items():
        df = df.drop(columns=['CV', 'CV_cons', 'CV_gauge','RMD_cons'])
        dfs_q[label] = df  # Save the modified DataFrame back
dfs_q =append_RMD(dfs_q=dfs_q)
dfs_q = append_coeffVar(dfs_q=dfs_q)
cdfPlot_RMD(dfs_q=dfs_q, color_dict=color_dict, column_to_plot = 'RMD_cons')
#cdfPlot_RMD(dfs_q=dfs_q, color_dict=color_dict, column_to_plot = 'RMD_gauge')
plot_cdf_coeff(dfs_q=dfs_q, color_dict=color_dict, algos_to_plot=['sic4dvar', 'momma', 'hivdi', 'neobam', 'geobam', 'consensus', 'sad', 'metroman'])

In [ ]:
#SAVE FILTERED Q
for label, df in dfs_q.items():
    print(label, df.reach_id.nunique())

#SAVES AND CALLS DEPENDING...
# pd.DataFrame(dfs_q['Fast']).to_csv('/all_q_a_perm_filtered.csv', index=False)
# pd.DataFrame(dfs_q['Science']).to_csv('/all_q_b_perm_filtered.csv', index=False)
# pd.DataFrame(dfs_q['Continuous']).to_csv('/all_q_c_perm_filtered.csv', index=False)
# pd.DataFrame(dfs_q['Sampled']).to_csv('/all_q_e_perm_filtered.csv', index=False)

# # CALL FILTERED Q
# fast = pd.read_csv('/all_q_a_perm_filtered.csv', low_memory=False)
# science = pd.read_csv('/all_q_b_perm_filtered.csv', low_memory=False)
# continuous = pd.read_csv('/all_q_c_perm_filtered.csv', low_memory=False)
# sampled = pd.read_csv('/all_q_e_perm_filtered.csv', low_memory=False)

In [ ]:
# Create dictionary to run all analyses across runs
# Input for almost all functions

df_list = [fast, science, continuous] #, runE_perm_clean] #, runD_perm_clean] 
df_list_common = subset_to_common_reach_ids(df_list)
print(df_list_common[0].reach_id.nunique())


dfs_q = {'Fast': df_list_common[0], 'Science': df_list_common[1], 'Continuous': df_list_common[2], 'Sampled': sampled}

for label, df in dfs_q.items():
    df['algo'] = df['algo'].replace({'geobam': 'neobam'})
    print(label, df.reach_id.nunique())

## Hist of FS Orbit Q Frequency

In [ ]:
df_filt = dfs_q['Continuous'].copy()

# Ensure 'time' is datetime
df_filt['time'] = pd.to_datetime(df_filt['time'])
df_filt = df_filt[df_filt['algo'] == 'consensus']
df_filt = df_filt.dropna(subset=['Q'])

cutoff_date = pd.Timestamp('2023-07-11')

# Extract month and year
df_filt['month'] = df_filt['time'].dt.month
df_filt['year'] = df_filt['time'].dt.year

# Label whether a record is FS Orbit (before cutoff) or SO years
def assign_group(row):
    if row['time'] < cutoff_date:
        return 'FS_2023'
    elif row['year'] == 2023:
        return 'SO_2023'
    elif row['year'] == 2024:
        return 'SO_2024'
    elif row['year'] == 2025:
        return 'SO_2025'
    else:
        return None

df_filt['group'] = df_filt.apply(assign_group, axis=1)
df_filt = df_filt.dropna(subset=['group'])
print(df_filt.groupby('group')['reach_id'].nunique())
print(df_filt[df_filt['group'] == 'FS_2023'].shape[0] / df_filt.shape[0])
# Count Q per reach, per month, per group
reach_monthly_counts = (
    df_filt.groupby(['reach_id', 'month', 'group'])['Q']
    .count()
    .reset_index(name='Q_count')
)

# Average across reaches for each (month, group)
avg_q_per_reach = (
    reach_monthly_counts
    .groupby(['month', 'group'])['Q_count']
    .mean()
    .reset_index()
)



# Pivot to wide format for grouped bar chart
pivot_df = avg_q_per_reach.pivot(index='month', columns='group', values='Q_count').fillna(0)

# Order columns (bars per month)
order = ['FS_2023', 'SO_2023', 'SO_2024', 'SO_2025']
pivot_df = pivot_df.reindex(columns=order, fill_value=0)

# Define custom colors (lightest to darkest blue)
colors = ['black',
          'dimgrey',  # light blue (FS Orbit)
          'grey',  # steel blue (2023_SO)
          'silver',  # dark blue (2024_SO)
        ]  # very dark navy (2025_SO)

# Plot grouped bar chart
ax = pivot_df.plot(kind='bar', figsize=(12, 8), width=0.8, color=colors, edgecolor='black')

plt.xlabel('Month', fontsize=30)
plt.ylabel('Average Q Instances per Reach', fontsize=30)
plt.title('Average Monthly Q per Reach by Year', fontsize=32)
plt.yticks(fontsize=26)
plt.xticks(
    ticks=range(0, 12),
    labels=['Jan','Feb','Mar','Apr','May','Jun',
            'Jul','Aug','Sep','Oct','Nov','Dec'],
    rotation=0,
    fontsize=26
)
plt.legend(title='Time Period', fontsize=26, title_fontsize=28)
plt.tight_layout()
plt.savefig('/figs/month_freq_poster.png', bbox_inches='tight', dpi=350)

plt.show()


In [ ]:
# Post filtering reach ids

# # Count how many algos each reach_id has
# algo_counts_per_reach = dfs_q['Continuous'].groupby('reach_id')['algo'].nunique()

# Count frequency per algorithm
df_bar = dfs_q['Continuous']
df_bar['algo'] = df_bar['algo'].replace({'geobam': 'neobam'})
algo_counts = df_bar.groupby('algo')['reach_id'].nunique()
# Plot
plt.figure(figsize=(10,6))
bars = plt.bar(algo_counts.index, algo_counts.values, color=[color_dict[a] for a in algo_counts.index])

# Add counts on top of bars
for bar in bars:
    plt.text(
        bar.get_x() + bar.get_width()/2, 
        bar.get_height(), 
        str(int(bar.get_height())), 
        ha='center', va='bottom'
    )

plt.ylim([0, 11500])
plt.ylabel('Reaches', fontsize=28)
plt.xlabel('Algorithm', fontsize=28)
plt.title('Algorithm Frequency (Continuous run)', fontsize=30)
plt.xticks(rotation=30, fontsize=26)
plt.yticks(fontsize=26)

plt.tight_layout()
plt.savefig('/figs/algoFreq_barChart.png', dpi=350)
plt.show()

In [ ]:
###############
# REVISIT TIME
###############

color_dict_timeseries = {
    'Continuous': '#D83A34',
    'Fast': '#FD8500',
    'Science': '#2B9EB3',
    'Sampled': '#6610F2',
    'gauge': 'darkgrey',
    'Continuous-FSO': '#FFEE32',
    'Continuous-SO': 'turquoise',
}

#GOOD TO GET HYDROCRON DATA FOR YOUR REACHES HERE
fsOrbit = pd.read_csv('/reachHydrocronData.csv')
print(fsOrbit.columns.values.tolist())
dfs_q_revisit = {}
for label, df in dfs_q.items():
    df_geo = df.merge(fsOrbit.drop(columns=['time', 'time_str']), how = 'left', on = ['reach_id'])
    dfs_q_revisit[label] = df_geo 

get_revisit_times(dfs_q=dfs_q_revisit, color_dict_timeseries=color_dict_timeseries)
get_revisit_times_overall(dfs_q=dfs_q_revisit, color_dict_timeseries=color_dict_timeseries)


## Plot the Orbit Timeseries

In [ ]:

labels = ['Fast', 'Science', 'Continuous', 'Sampled', 'gauge']

color_dict_timeseries = {
    'Continuous': '#D83A34',
    'Fast': '#FD8500',
    'Science': '#2B9EB3',
    'Sampled': '#6610F2',
    'gauge': 'black'
}

plot_consensus_from_multiple_dfs(
    dfs_dict=dfs_q,
    labels=labels,
    divide_date=pd.to_datetime('2023-07-11'),
    color_dict=color_dict_timeseries,
    algo='consensus'
)


## Q Difference Boxplots with Significance

### By Continent

In [ ]:
# by reach
plot_seasonal_log_ratio_boxplots_by_continent_reach(dfs_dict=dfs_q, consensus_algo='consensus', plot=True)

In [ ]:
fast_gaugeFullRange = pd.read_csv('/dfs_gauge_fullRange_a_perm_filtered.csv')
science_gaugeFullRange = pd.read_csv('/dfs_gauge_fullRange_b_perm_filtered.csv')
continuous_gaugeFullRange = pd.read_csv('/dfs_gauge_fullRange_c_perm_filtered.csv')
sampled_gaugeFullRange= pd.read_csv('/dfs_gauge_fullRange_e_perm_filtered.csv')

dfs_gaugeFullRange = {'Fast': fast_gaugeFullRange, 'Science': science_gaugeFullRange, 'Continuous': continuous_gaugeFullRange, 'Sampled': sampled_gaugeFullRange}

# Add continuous orbit split
runC_perm_fast_gaugeFullRange = dfs_gaugeFullRange['Continuous'][pd.to_datetime(dfs_gaugeFullRange['Continuous']['time']) < divide_date]
runC_perm_science_gaugeFullRange = dfs_gaugeFullRange['Continuous'][pd.to_datetime(dfs_gaugeFullRange['Continuous']['time']) >= divide_date]

dfs_gaugeFullRange['Continuous-FSO'] = runC_perm_fast_gaugeFullRange
dfs_gaugeFullRange['Continuous-SO'] = runC_perm_science_gaugeFullRange

print(dfs_gaugeFullRange['Fast'].algo.unique())

    
# Add continuous orbit split
runC_perm_fast = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) < divide_date]
runC_perm_science = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) >= divide_date]

dfs_q['Continuous-FSO'] = runC_perm_fast
dfs_q['Continuous-SO'] = runC_perm_science

In [ ]:
# Compare gauged and ungauged discharge

plot_seasonal_log_ratio_by_continent_reach_gauged_vs_ungauged(
    dfs_dict_gauged=dfs_gaugeFullRange,
    dfs_dict_ungauged=dfs_q,
    consensus_algo='consensus',
    output_dir='/figs/'
)


### Regime Characteristics

In [ ]:

# Run the function
aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_q, algo='consensus')

# Combine and save
overall_summary_all_cons = pd.concat(aggregated_q_overall.values(), ignore_index=True)
overall_summary_all_cons.to_csv('/consensus_regime_summary.csv', index=False)
print('save successful')


# aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_gauge_fullRange, algo='gauge')

# # Combine and save
# aggregated_q_overall_gauge_fullRange = pd.concat(aggregated_q_overall.values(), ignore_index=True)
# aggregated_q_overall_gauge_fullRange.to_csv('/gauge_gauges_regime_summary.csv', index=False)
# print('save successful')

# aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_gauge_fullRange, algo='gauge_swot_match')

# # Combine and save
# overall_summary_all_gauge_swot = pd.concat(aggregated_q_overall.values(), ignore_index=True)
# overall_summary_all_gauge_swot.to_csv('/gauge_swot_match_gauges_regime_summary.csv', index=False)
# print('save successful')


aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_q, algo='metroman')

# Combine and save
overall_summary_all_metroman = pd.concat(aggregated_q_overall.values(), ignore_index=True)
overall_summary_all_metroman.to_csv('/metroman_regime_summary.csv', index=False)
print('save successful')

aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_q, algo='sic4dvar')

# Combine and save
overall_summary_all_sic4dvar = pd.concat(aggregated_q_overall.values(), ignore_index=True)
overall_summary_all_sic4dvar.to_csv('/sic4dvar_regime_summary.csv', index=False)
print('save successful')


aggregated_q_overall = summarize_overall_Q(dfs_q=dfs_q, algo='momma')

# Combine and save
overall_summary_all_momma = pd.concat(aggregated_q_overall.values(), ignore_index=True)
overall_summary_all_momma.to_csv('/momma_regime_summary.csv', index=False)
print('save successful')

In [ ]:
overall_summary_all_cons = pd.read_csv('/consensus_regime_summary.csv')
overall_summary_all_metroman = pd.read_csv('/metroman_regime_summary.csv')
overall_summary_all_sic4dvar = pd.read_csv('/sic4dvar_regime_summary.csv')
overall_summary_all_momma = pd.read_csv('/momma_regime_summary.csv')
overall_summary_all_neobam = pd.read_csv('/neobam_regime_summary.csv')

overall_summary = pd.concat([overall_summary_all_cons,overall_summary_all_metroman,overall_summary_all_sic4dvar,overall_summary_all_momma])
print(overall_summary.head())

#### Overall Algo by orbit

In [ ]:


color_dict = {
    'sic4dvar': 'green',
    'momma': 'blue',
    'neobam': 'purple',
    'consensus': 'sienna',
    'metroman': 'orange',
    'geobam': 'purple',
    'hivdi': 'deeppink',
    'sad': 'tomato',
    'gauge': 'black'
}

algos = ['momma', 'sic4dvar', 'metroman', 'neobam', 'geobam', 'consensus']
orbit_pairs = [
    ('Science', 'Continuous'),
    ('Fast', 'Continuous'),
    ('Fast', 'Sampled'),
]

plot_logQ_diff_grouped_by_orbit_combo_with_stats(
    dfs_dict=dfs_q,
    algos=algos,
    orbit_pairs=orbit_pairs,
    color_dict=color_dict
)


## Reach Comparison - Q Distribution

In [ ]:

from scipy.stats import ks_2samp, wasserstein_distance



color_dict_runCombos = {
    'Continuous': '#D83A34',
    'Fast': '#FD8500',
    'Science': '#2B9EB3',
    'Sampled': '#6610F2',
    'gauge': 'black',
    'Continuous-FSO': '#FFEE32',
    'Continuous-SO': 'turquoise',
}

# Add continuous orbit split
runC_perm_fast = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) < divide_date]
runC_perm_science = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) >= divide_date]

dfs_q['Continuous-FSO'] = runC_perm_fast
dfs_q['Continuous-SO'] = runC_perm_science


summary_metrics = plot_reach_consensus_cdfs(
    df_dict=dfs_q,
    variable='Q',
    algo='consensus',
    color_dict=color_dict_runCombos
)

summary_metrics

In [ ]:
# This takes awhile

color_dict_runCombos = {
    'Continuous': '#D83A34',
    'Fast': '#FD8500',
    'Science': '#2B9EB3',
    'Sampled': '#6610F2',
    'gauge': 'black',
    'Continuous-FSO': '#FFEE32',
    'Continuous-SO': 'blue',
}

# Add continuous orbit split
runC_perm_fast = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) < divide_date]
runC_perm_science = dfs_q['Continuous'][pd.to_datetime(dfs_q['Continuous']['time']) >= divide_date]

dfs_q['Continuous-FSO'] = runC_perm_fast
dfs_q['Continuous-SO'] = runC_perm_science

# # 
summary_metrics = plot_reach_consensus_cdfs(
    df_dict=dfs_q,
    variable='Q',
    algo='consensus',
    color_dict=color_dict_runCombos
)

summary_metrics

In [ ]:
#Join to FS reaches
fsOrbit = pd.read_csv('/figs/fsOrbit_hydrocron.csv')

summary_metrics_geo = summary_metrics.merge(fsOrbit, how = 'left', on = ['reach_id'])

print(summary_metrics_geo.shape)
print(summary_metrics_geo)
summary_metrics_geo.to_csv('/figs/summary_reach_metrics_orbit_geo.csv', index=False)


In [ ]:
summary_metrics_geo = pd.read_csv('/figs/summary_reach_metrics_orbit_geo.csv')
print(summary_metrics_geo.columns.values.tolist())

In [ ]:
#Summary metric plots

plot_pair_stacked_bars(df=summary_metrics_geo, metric='nBIAS')

### by region

In [ ]:
color_dict_runCombos = {
    'Continuous': '#D83A34', #red
    'Fast': '#FD8500', #Orange
    'Science': '#2B9EB3', #blue
    'Sampled': '#6610F2', #purple
    'gauge': 'black',
    'Continuous-FSO': '#FFEE32', #yellow
    'Continuous-SO': 'blue', #blue
}

In [ ]:
plot_pair_by_continent(df=summary_metrics_geo, pairs_to_plot=["Fast-Continuous-FSO", "Science-Continuous-SO"], metric='nBIAS')